In [79]:
import pandas as pd
import numpy as np

import requests
import re
import io

from datetime import timedelta

In [2]:
df = pd.read_csv('../../data/processed/wildfire.csv')

df['FireDiscoveryDateTime'] = pd.to_datetime(df['FireDiscoveryDateTime'], infer_datetime_format=True, errors = 'coerce')
df['ControlDateTime'] = pd.to_datetime(df['ControlDateTime'], infer_datetime_format=True, errors = 'coerce')

In [33]:
def build_request_param(id, lat, long, start_date, end_date, result_df, base_uri):
    params = f"latitude={lat}&longitude={long}&start={start_date.strftime('%Y%m%d')}&end={end_date.strftime('%Y%m%d')}"
    base_uri += params

    res = requests.get(base_uri)

    split_text = res.text.split("-END HEADER-")

    if len(split_text) >= 2:
        response_df = pd.read_csv(io.StringIO(split_text[1]))
        response_df["LAT"] = lat
        response_df["LONG"] = long
        response_df["PID"] = id
        result_df = pd.concat([result_df, response_df])
        return result_df
        
    return result_df

In [28]:
nasa_df = pd.DataFrame()
parameters = "parameters=T2M,T2M_MAX,QV2M,PRECTOTCORR,WS2M,WS2M_MAX,WS10M,WS10M_MAX,GWETTOP,GWETPROF"
base_uri = f"https://power.larc.nasa.gov/api/temporal/daily/point?time-standard=lst&header=true&format=csv&community=sb&{parameters}&"

for i in df.index:
    nasa_df = build_request_param(
        i,
        df.loc[i, "InitialLatitude"],
        df.loc[i, "InitialLongitude"],
        df.loc[i, "FireDiscoveryDateTime"],
        df.loc[i, "ControlDateTime"],
        nasa_df,
        base_uri,
    )

In [30]:
nasa_df.to_csv("../../data/raw/nasa_weather.csv", index=False)


Get relative humidity to calculate fuel moister

In [24]:
filtered_df = (df[df['DailyAcres'] > 1]) 
filtered_df.shape

filtered_df = filtered_df[((filtered_df['ControlDateTime'] - filtered_df['FireDiscoveryDateTime']) > timedelta(days=1))]
filtered_df.shape

(2863, 20)

In [54]:
filtered_df.head(1)

,X,Y,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,FireCause,FireDiscoveryDateTime,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,IrwinID,LocalIncidentIdentifier,POOCounty,POODispatchCenterID,POOFips,POOState,UniqueFireIdentifier,id
1,-115.748812,40.617506,2020-08-03 23:00:00+00:00,2020-09-02 15:00:00+00:00,5985.9,5.0,Natural,2020-07-19 23:00:00+00:00,WF,FI,40.602563,-115.719777,{91E0CBAB-A24E-4590-B6C6-2B4A46907E8A},10145,Elko,NVEIC,32007,US-NV,2020-NVECFX-010145,1


In [34]:
nasa_df = pd.DataFrame()
parameters = "parameters=T2MDEW,T2MWET,RH2M,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_UV_INDEX"
base_uri = f"https://power.larc.nasa.gov/api/temporal/daily/point?time-standard=lst&header=true&format=csv&community=sb&{parameters}&"

for i in filtered_df.index:
    nasa_df = build_request_param(
        filtered_df.loc[i, 'id'],
        filtered_df.loc[i, "InitialLatitude"],
        filtered_df.loc[i, "InitialLongitude"],
        filtered_df.loc[i, "FireDiscoveryDateTime"],
        filtered_df.loc[i, "ControlDateTime"],
        nasa_df,
        base_uri,
    )

In [35]:
nasa_df.head(1)

,YEAR,MO,DY,T2MDEW,T2MWET,RH2M,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_UV_INDEX,LAT,LONG,ID
0,2020,7,19,3.02,13.91,27.31,155.74,135.5,2.55,40.602563,-115.719777,1


In [ ]:
nasa_df.rename(columns={
  'T2MDEW': 'dew_frost_point_2m',
  'T2MWET': 'wet_bulb_temp_2m',
  'RH2M': 'relative_humidity',
  'CLRSKY_SFC_PAR_TOT': 'clear_sky_photosynthetically_active_radiation',
  'ALLSKY_SFC_PAR_TOT': 'all_sky_photosynthetically_active_radiation',
  'ALLSKY_SFC_UV_INDEX': 'all_sky_surface_uv_index'
}, inplace=True)

In [36]:
nasa_df.to_csv("../../data/raw/nasa_weather_extra.csv", index=False)


----
Past 6 months precipitation for filtered fire

In [76]:
def build_request_param(id, lat, long, start_date, end_date, result_df, base_uri):
    params = f"latitude={lat}&longitude={long}&start={start_date.strftime('%Y')}&end={end_date.strftime('%Y')}"
    base_uri += params

    res = requests.get(base_uri)

    split_text = res.text.split("-END HEADER-")

    if len(split_text) >= 2:
        response_df = pd.read_csv(io.StringIO(split_text[1]))
        response_df["PID"] = id
        result_df = pd.concat([result_df, response_df])
        return result_df
        
    return result_df

In [77]:
nasa_last_180_prec_df = pd.DataFrame()
parameters = "parameters=PRECTOTCORR_SUM,PRECSNO"
base_uri = f"https://power.larc.nasa.gov/api/temporal/monthly/point?header=true&format=csv&community=sb&{parameters}&"

for i in filtered_df.index:
    nasa_last_180_prec_df = build_request_param(
        filtered_df.loc[i, 'id'],
        filtered_df.loc[i, "InitialLatitude"],
        filtered_df.loc[i, "InitialLongitude"],
        filtered_df.loc[i, "FireDiscoveryDateTime"] - pd.tseries.offsets.DateOffset(months=6),
        filtered_df.loc[i, "FireDiscoveryDateTime"],
        nasa_last_180_prec_df,
        base_uri,
    )

test_df = build_request_param(
        filtered_df.loc[1, 'id'],
        filtered_df.loc[1, "InitialLatitude"],
        filtered_df.loc[1, "InitialLongitude"],
        filtered_df.loc[1, "FireDiscoveryDateTime"] - pd.tseries.offsets.DateOffset(months=12),
        filtered_df.loc[1, "FireDiscoveryDateTime"],
        nasa_last_180_prec_df,
        base_uri,
    )

In [80]:
test_df.head()


,PARAMETER,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN,ID
0,PRECSNO,2020,1.32,0.66,0.66,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.66,0.66,0.00,1
1,PRECTOTCORR_SUM,2020,52.73,21.09,26.37,10.55,21.09,26.37,0.0,0.00,5.27,0.0,31.64,21.09,216.21,1
0,PRECSNO,2020,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,5
1,PRECTOTCORR_SUM,2020,0.00,21.09,63.28,5.27,0.00,0.00,0.0,5.27,0.00,0.0,5.27,0.00,100.20,5
0,PRECSNO,2019,0.66,1.98,0.66,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1.32,0.66,10


In [56]:
filtered_df.loc[1, "FireDiscoveryDateTime"] - pd.tseries.offsets.DateOffset(months=6),

(Timestamp('2020-01-19 23:00:00+0000', tz='UTC'),)

In [51]:
nasa_last_180_prec_df.head(1)

""


In [82]:
nasa_last_180_prec_df.to_csv("../../data/raw/nasa_weather_last_180days.csv", index=False)
